In [1]:
import networkx as nx
import os
from utils.global_rules import rules

dataDir = "../data/"
cacheDir = "../data/cache/"

读入图数据，G: 无向图，G_directed: 有向图


In [2]:
import importlib

process = importlib.import_module("process")
importlib.reload(process)

G = process.get_graph()

Loading graph...
Initial Node shape:  (2371558, 4)
Initial Link shape:  (3286986, 3)
After drop nan and duplicates:
Node shape:  (2371558, 4)
Link shape:  (3285490, 3)
Undirected graph loaded.


In [23]:
# Create a Directed Graph from the undirected one
G_directed = nx.DiGraph(G)

运行社区检测


In [3]:
import importlib

community_detection = importlib.import_module("utils.community_detection")
importlib.reload(community_detection)

# community_detection.get_community_for_all_groups(G)
community_detection.get_subgraph_for_all_communities(G)

根据以上社区检测算法的结果，选取团伙 [?] 进行分析。


In [44]:
import json
import importlib
import pandas as pd

rules = importlib.import_module("utils.global_rules")
process = importlib.import_module("process")
importlib.reload(rules)
importlib.reload(process)

rules = rules.rules
# group_to_analyze = ["1", "3", "4"]
group_to_analyze = ["1", "2", "3", "4", "5"]
# group_to_analyze = ["1", "2", "4", "5"]
cacheDir = "../data/cache/"
subgraphDir = "../data/subgraph/"
tempDir = "../data/temp/"
answerDir = "../data/answer/"

params = {
    "1": {
        "pagerankQuantile": 0.95,
        "degreeCentralityQuantile": 0.95,
        "degreeQuantile": 0.8,
        "jump_limit": rules["jump_limit_specified_by_industry"]["1"],
        "countKeepPercent": 0.1,
    },
    "2": {
        "pagerankQuantile": 0.92,
        "degreeCentralityQuantile": 0.92,
        "degreeQuantile": 0.9,
        "jump_limit": rules["jump_limit_specified_by_industry"]["2"],
        "countKeepPercent": 0.6,
    },
    "3": {
        "pagerankQuantile": 0.95,
        "degreeCentralityQuantile": 0.95,
        "degreeQuantile": 0.94,
        "jump_limit": rules["jump_limit_specified_by_industry"]["3"],
        "countKeepPercent": 0.4,
    },
    "4": {
        "pagerankQuantile": 0.9,
        "degreeCentralityQuantile": 0.88,
        "degreeQuantile": 0.82,
        "jump_limit": rules["jump_limit_specified_by_industry"]["4"],
        "countKeepPercent": 0.4,
    },
    "5": {
        "pagerankQuantile": 0.7,
        "degreeCentralityQuantile": 0.7,
        "degreeQuantile": 0.85,
        "jump_limit": rules["jump_limit_specified_by_industry"]["5"],
        "countKeepPercent": 0.4,
    },
}


def export_subgraph(graph, group_str: str):
    # transform the graph to directed graph
    # graph = graph.to_directed()
    # export the graph to json
    data = nx.readwrite.json_graph.node_link_data(graph)
    # Keep only the nodes and edges
    data = {"nodes": data["nodes"], "edges": data["links"]}
    json.dump(
        data,
        open(tempDir + "group_" + group_str + ".json", "w"),
        indent=4,
    )

    json.dump(
        nx.readwrite.json_graph.node_link_data(graph),
        open(subgraphDir + "group_" + group_str + ".json", "w"),
        indent=4,
    )

    # Create a directory in subgraphDir
    if not os.path.exists(subgraphDir + group_str):
        os.makedirs(subgraphDir + group_str)

    node_data = pd.DataFrame(data["nodes"])
    node_data.to_csv(subgraphDir + group_str + "/nodes.csv", index=False)
    link_data = pd.DataFrame(data["edges"])
    link_data.to_csv(subgraphDir + group_str + "/links.csv", index=False)

    if not os.path.exists(answerDir + group_str):
        os.makedirs(answerDir + group_str)

    # export node in format: id, name, type, industry
    node_data[["id", "name", "type", "industry"]].to_csv(
        answerDir + group_str + "/nodes.csv", index=False
    )
    # export link in format: relation, source, target
    link_data[["relation", "source", "target"]].to_csv(
        answerDir + group_str + "/links.csv", index=False
    )

    return


def analyse_group(group_str: str):
    print("analyse group: ", group_str)
    # get the evidence
    evidence = rules["evidence"][group_str]

    # # read the group
    # graph = nx.readwrite.json_graph.node_link_graph(
    #     json.load(open(cacheDir + "group_" + group_str + ".json"))
    # )
    # # print graph info
    # print("#nodes: ", graph.number_of_nodes())
    # print("#edges: ", graph.number_of_edges())
    # subgraph = process.get_subgrah(graph, evidence)

    subgraph = process.get_subgraph(G, evidence, params[group_str]["jump_limit"])

    subgraph_filtered = process.filter_subgraph(
        subgraph,
        pagerankQuantile=params[group_str]["pagerankQuantile"],
        degreeCentralityQuantile=params[group_str]["degreeCentralityQuantile"],
        degreeQuantile=params[group_str]["degreeQuantile"],
        countKeepPercent=params[group_str]["countKeepPercent"],
        scaleThreshold=rules["net_limit"][rules["scale"][group_str]]["node"],
        # verbose=True,
    )
    process.set_core(subgraph_filtered)
    export_subgraph(subgraph_filtered, group_str)

    print()
    return


for group in group_to_analyze:
    analyse_group(group)

# analyse_group("2")

analyse group:  1
#nodes in subgraph:  33821
#edges in subgraph:  33974
Nodes left:  199
Edges left:  296
Nodes with non-empty industry: 131 / 182

analyse group:  2
#nodes in subgraph:  5125
#edges in subgraph:  5853
Nodes left:  504
Edges left:  868
Nodes with non-empty industry: 328 / 475

analyse group:  3
#nodes in subgraph:  36149
#edges in subgraph:  37891
Nodes left:  638
Edges left:  1318
Nodes with non-empty industry: 119 / 350

analyse group:  4
#nodes in subgraph:  27268
#edges in subgraph:  32401
Nodes left:  2342
Edges left:  4609
Nodes with non-empty industry: 800 / 2074

analyse group:  5
#nodes in subgraph:  18534
#edges in subgraph:  20714
Nodes left:  1912
Edges left:  3223
Nodes with non-empty industry: 995 / 1728



In [112]:
import importlib

count = importlib.import_module("utils.count")
importlib.reload(count)

count.count_type_for_all_groups(group_to_analyze)

group:  1
group:  2
group:  3
group:  4
group:  5
